In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, TimeoutException, WebDriverException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from apscheduler.schedulers.background import BackgroundScheduler

from contextlib import contextmanager

from rich.console import Console
from rich.layout import Layout
from rich.table import Table
from rich.live import Live
from rich import box

from getpass import getpass

from datetime import datetime, timedelta, timezone
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import re
import random
import requests
import subprocess
import time

console = Console()

input_site = 'https://ts31.x3.international.travian.com/dorf1.php'
SERVER_SPEED = 3

REFRESH = 'refresh'
RESOURCE_FIELDS = 'resource_fields'
ADVENTURES = 'adventures'
HERO_UPGRADE = 'hero_upgrade'
GOLD_CLUB_CHECK = 'gold_club_check'
COLLECT_MISSION_RESOURCES = 'collect_mission_resources'
COLLECT_DAILY_QUEST_REWARDS = 'collect_daily_rewards'
TRAIN_TROOPS = 'train_troops'
CHECK_FOR_INCOMING_ATTACKS = 'attack_check'
SPEND_ALL_RESOURCES = 'spend_all'
RAIDS = 'raids'
logs = {}

WOOD = 1
CLAY = 2
IRON = 3
WHEAT = 4
resource_ids = [WOOD, CLAY, IRON, WHEAT]

JS_CLICK = "arguments[0].click();"
JS_SCROLL_INTO_VIEW = "arguments[0].scrollIntoView(true);"


def calc_interval_for_server_speed(x, server_speed=1):
    lower_bound_diff = random.randint(0, 45)
    upper_bound_diff = random.randint(0, 45)
    return random.uniform((x - lower_bound_diff) / SERVER_SPEED, (x + upper_bound_diff) / SERVER_SPEED)


# helper funcs
def calc_interval_around(x):
    lower_bound_diff = random.randint(0, 45)
    upper_bound_diff = random.randint(0, 45)
    return random.uniform(x - lower_bound_diff, x + upper_bound_diff)


def calc_new_interval_between(x, y):
    return random.uniform(x, y)


def refresh_page(drivers_info, driver, scheduler=None):
    driver.refresh()

    if scheduler:
        scheduler.add_job(refresh_page, 'interval',
                          seconds=calc_interval_for_server_speed(644, server_speed=SERVER_SPEED),
                          id=f"{drivers_info[driver]['Username']}_{REFRESH}",
                          args=[drivers_info, driver, scheduler], replace_existing=True)

In [3]:
''' web driver init. '''

def run_command_in_background(command):
    return subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=0)


def run_command_in_background_and_wait_for_output(command, expected_output, timeout=30):
    process = run_command_in_background(command)

    end_time = time.time() + timeout
    while time.time() < end_time:
        output = process.stdout.read(1024).decode('utf-8')

        if expected_output in output:
            print("Expected output received.")
            return True

        time.sleep(1)

    print("Timeout or process ended without producing expected output.")
    return False


def init_webdriver(proxy_port=None):
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--ignore-certificate-errors")

    # Disable image loading
    #prefs = {"profile.managed_default_content_settings.images": 2}
    #options.add_experimental_option("prefs", prefs)

    if proxy_port:
        proxy_address = f'localhost:{proxy_port}'
        options.add_argument(f'--proxy-server=http://{proxy_address};https://{proxy_address}')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        driver.get('https://httpbin.org/ip')
        return driver
    except WebDriverException:
        run_command_in_background_and_wait_for_output('proxy-manager', 'Proxy Manager is running')
        
        try:
            driver.refresh()         
            return driver
        except WebDriverException:
            return None
        
# Context manager for Selenium Web Driver
@contextmanager
def managed_webdriver(proxy_port=None):
    driver = init_webdriver(proxy_port)
    try:
        yield driver
    finally:
        print("Shutting down web driver...")
        driver.quit()

In [4]:
''' scheduler init. '''

def cleanup_scheduler(scheduler):
    if scheduler.running:
        print("Shutting down scheduler...")
        scheduler.shutdown(wait=True)
    else:
        print("Scheduler already stopped.")

# Context manager for BackgroundScheduler
@contextmanager
def managed_scheduler(*args, **kwargs):
    scheduler = BackgroundScheduler(*args, **kwargs)
    scheduler.start()
    try:
        yield scheduler
    finally:
        cleanup_scheduler(scheduler)

In [5]:
''' contextual help (dialog) helpers'''

def disable_contextual_help(driver):
    button_options = driver.find_element(By.XPATH, '//a[contains(@href, "/options")]')
    driver.execute_script(JS_CLICK, button_options) # use js to get around helper popup

    checkbox_contextual_help = driver.find_element(By.XPATH, '//*[@id="hideContextualHelp"]')
    checkbox_contextual_help.click()
    
    save_button = driver.find_element(By.CSS_SELECTOR, '.textButtonV1.green')
    driver.execute_script(JS_SCROLL_INTO_VIEW, save_button)
    save_button.click()


def dismiss_report_helper_popup(driver):
    button_reports = driver.find_element(By.XPATH, '//*[@id="navigation"]/a[5]')
    driver.execute_script(JS_CLICK, button_reports) # use js to get around helper popup


def dismiss_ok_popup(driver):
    try:
        button_ok = driver.find_element(By.XPATH, '//*[@id="contextualHelp"]/div/div[2]/nav/button')
        driver.execute_script(JS_CLICK, button_ok) # use js to get around helper popup
    except NoSuchElementException:
        pass


def dismiss_popup(driver):
    try:
        popup = driver.find_element(By.ID, 'contextualHelp')
    except NoSuchElementException:
        return
    
    steps_description = driver.find_element(By.CSS_SELECTOR, '.steps').text
    if "You received a new report." in steps_description:
        dismiss_report_helper_popup(driver)
    elif "You're safe for now." in steps_description:
        dismiss_ok_popup(driver)


def dismiss_deal(driver):
    try:
        #TODO: FINISH TESTING THIS
        button_dismiss = driver.find_element(By.XPATH, '//*[starts-with(@id, "oneTimeOfferAnnouncement")]')
        button_dismiss.click()
    except NoSuchElementException:
        pass


In [6]:
''' Open Travian International 2 server and login if not already '''

def attempt_login(site, username, password, driver, retry_attempts=1):
    driver.get(site)

    try:
        input_username = WebDriverWait(driver, 7).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="loginForm"]/tbody/tr[1]/td[2]/input'))
        )
        input_username.send_keys(username)
        
        input_pwd = driver.find_element(By.XPATH, '//*[@id="loginForm"]/tbody/tr[2]/td[2]/input')
        input_pwd.send_keys(password)

        button_login = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"][value="Login"].textButtonV1.green')
        button_login.click()

        #driver.refresh()
    except NoSuchElementException:
        driver.refresh()

        if retry_attempts > 0:
            attempt_login(site, username, password, driver, retry_attempts-1)
            return
    
    dismiss_deal(driver) # TODO: TESTING NOW

In [7]:
''' navigation helpers '''

def navigate_to_page(driver, page_css_selector, url_check):
    dismiss_popup(driver)

    if url_check is None or url_check not in driver.current_url:
        try:
            button_page = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, page_css_selector))
            )
        except TimeoutException:
            # if page button not found, navigate to resource field page as default and try again
            driver.get(driver.current_url.split('.com')[0] + '.com/dorf1.php')
            button_page = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, page_css_selector))
            )
        
        if button_page:
            try:
                button_page.click()
            except ElementClickInterceptedException:
                driver.execute_script(JS_CLICK, button_page)        


def navigate_to_hero_inventory(driver):
    selector = '#heroImageButton'
    url_check = 'hero/inventory'
    navigate_to_page(driver, selector, url_check)


def navigate_to_resource_fields(driver):
    selector = '.village.resourceView'
    url_check = 'dorf1.php'
    navigate_to_page(driver, selector, url_check)


def navigate_to_buildings(driver):
    selector = '.village.buildingView'
    url_check = 'dorf2.php'
    navigate_to_page(driver, selector, url_check)


def enter_building(driver, building):
    if building is None or pd.isna(building):
        print("Troop Building is undefined!")
        return

    navigate_to_buildings(driver)

    selector = f'[data-name="{building}"]'
    url_check = None
    try:
        navigate_to_page(driver, selector, url_check)
    except TimeoutException:
        print("Building does not exist!")

In [8]:
''' building construction funcs '''

def can_build(building_slot):
    if 'good' not in building_slot.get_attribute('class'):
        print("Unable to upgrade, please ensure you have enough resources and building queue is not full.")
        return False
    return True


def press_construct_building_button_for(driver, building):
    try:
        header = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
            (By.XPATH, f"//h2[text()='{building}']"))
        )
    except TimeoutException:
        print("Unable to find building! Has it already been constructed or are you on the wrong building tab?")
        return False

    building_wrapper = header.find_element(By.XPATH, '..')
    build_button = building_wrapper.find_element(By.CSS_SELECTOR, '.textButtonV1.green.new')
    build_button.click()

    return True

def construct_building_in_slot(driver, building, building_slot):
    navigate_to_buildings(driver)

    building_slot = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
        (By.XPATH, f"//a[contains(@href, '/build.php?id={building_slot}')]"))
    )
    if not can_build(building_slot):
        return

    driver.execute_script(JS_CLICK, building_slot) # use javascript to get around helper popup

    for i in range(1, 4): # check all building tabs for building
        try:
            infrastructure_tab = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
                (By.XPATH, f"//a[contains(@href, '/build.php?id={building_slot}&category={i}')]"))
            )

            infrastructure_tab.click()
        except TimeoutException: # if tab not found we have another helper popup, dismiss it then retry
            popup_next_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
                (By.XPATH, "//*[@id='contextualHelp']/div/div[2]/nav/button"))
            )

            # dismiss initial helper dialog
            driver.execute_script(JS_CLICK, popup_next_button)
            driver.execute_script(JS_CLICK, popup_next_button)

        began_constructing = press_construct_building_button_for(driver, building)
        if began_constructing:
            break

    print(f"Successfully began constructing {building} in slot {building_slot}")


def wall_built(driver):
    navigate_to_buildings(driver)

    building_slot = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@id='villageContent']/div[22]"))
    )

    if building_slot.get_attribute('data-name'):
        return True
    return False


def construct_wall(driver):
    navigate_to_buildings(driver)

    building_slot = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
        (By.XPATH, "//*[@id='villageContent']/div[22]"))
    )

    if not can_build(building_slot):
        return

    driver.execute_script(JS_CLICK, building_slot) # use javascript to get around helper popup

    # wall is the only building allowed in slot 40, build button for it will be the only one that exists on the page
    try:
        build_button = driver.find_element(By.CSS_SELECTOR, '.textButtonV1.green.new')
        build_button.click()
    except NoSuchElementException:
        print("Wall has already been constructed!")

In [9]:
def upgrade_building(driver, building):
    try:
        building = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, f'[data-name="{building}"]'))
        )
    except NoSuchElementException:
        print("Building does not exist! Unable to upgrade building.")

    if not can_build(building):
        return
    
    enter_building(driver, building)

    button_upgrade = driver.find_element(By.CSS_SELECTOR, '.textButtonV1.green.build')
    button_upgrade.click()


def upgrade_mission_clay_field(driver):
    navigate_to_resource_fields(driver)

    building_slot = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
        (By.XPATH, "//a[contains(@href, '/build.php?id=5')]"))
    )

    if not can_build(building_slot):
        return

    driver.execute_script(JS_CLICK, building_slot) # use javascript to get around helper popup

    button_upgrade = driver.find_element(By.CSS_SELECTOR, '.textButtonV1.green.build')
    if '2' in button_upgrade.text:
        button_upgrade.click()
    else:
        print("Clay field already at lvl 2.")


def complete_construction_immediately(driver):
    navigate_to_buildings(driver)

    button_complete_construction_immediately = WebDriverWait(driver, 7).until(EC.presence_of_element_located(
        (By.XPATH, '//button[contains(@value, "Distribute")]'))
    )
    time.sleep(3) # manually sleep to let time for button to be clickable
    button_complete_construction_immediately.click()


def run_missions_and_disable_contextual_helpers(driver):
    disable_contextual_help(driver)
    
    upgrade_mission_clay_field(driver)

    construct_wall(driver)
    
    construct_building_in_slot(driver, 'Cranny', 30)
    construct_building_in_slot(driver, 'Warehouse', 32)
    construct_building_in_slot(driver, 'Granary', 35)
    construct_building_in_slot(driver, 'Embassy', 24)

    upgrade_building(driver, 'Main Building')
    upgrade_building(driver, 'Main Building')

    upgrade_building(driver, 'Cranny')
    upgrade_building(driver, 'Cranny')
    
    construct_building_in_slot(driver, 'Barracks', 23)
    construct_building_in_slot(driver, 'Marketplace', 19)

    dismiss_report_helper_popup(driver)
    dismiss_ok_popup(driver)

In [10]:
# TODO: Func is error prone, revisit and reimplement
def get_resources_from_hero(driver, resource_type=-1):
    resources_to_get = [resource_type]
    if resource_type == -1:
        resources_to_get = [WOOD, CLAY, IRON, WHEAT]
    
    navigate_to_hero_inventory(driver)

    for resource_id in resources_to_get:
        amount_to_collect = 0

        # if we have all res types, max res index will be iron at position 4, no need to go beyond this
        for i in range(1, 5):
            item = WebDriverWait(driver, 12).until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, f'.heroItems.filter_all > div:nth-child({i})'))
            )
            item_type = item.find_element(By.XPATH, './div[1]').get_attribute('class')

            id_str = ''.join(filter(str.isdigit, item_type))
            item_id = int(id_str) if id_str else -1
            
            if item_id == 144 + resource_id: # if item ID matches resource type
                try:
                    item.click()
                except ElementClickInterceptedException:
                    continue

                try:
                    input_amount = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
                        (By.XPATH, '//*[@id="consumableHeroItem"]/label/input'))
                    )
                    amount = input_amount.get_attribute('value')
                except StaleElementReferenceException:
                    continue

                amount_to_collect = int(int(amount) * 0.64) # only get 64% of total amount to prevent filling warehouses

                # calling clear() or sending value of '0' through JS causes amount sent to be incorrect,
                # send backspace keystrokes manually
                input_amount.send_keys(Keys.END)  # Move cursor to the end of the text
                for _ in range(len(input_amount.get_attribute('value'))):
                    input_amount.send_keys(Keys.BACKSPACE)
                input_amount.send_keys(str(amount_to_collect))
                
                button_transfer = WebDriverWait(driver, 7).until(EC.presence_of_element_located(
                    (By.CSS_SELECTOR, '.textButtonV2.buttonFramed.rectangle.withText.green'))
                )
                #EC.element_to_be_clickable not functioning consistently, manually wait to ensure button is clickable
                time.sleep(5)
                button_transfer.click()

                time.sleep(0.7) # allow page time to load after transfer

                #print(f"Successfully collected resource {resource_id} (amount={amount_to_collect}) from hero.")
                break

        #print(f"Unable to collect resource {resource_id} from hero.")

In [11]:
''' resource field upgrade job '''

'''
def can_afford_resource_field_upgrade(driver):
    navigate_to_resource_fields(driver)

    resource_field_container = driver.find_element(By.XPATH, '//*[@id="resourceFieldContainer"]')
    resource_field_links = resource_field_container.find_elements(By.TAG_NAME, 'a')

    
    num_res_fields = len(resource_field_links)
    for i in range(2, num_res_fields + 1): # start at 2 to skip village center
        link = driver.find_element(By.XPATH, f'//*[@id="resourceFieldContainer"]/a[{i}]')
        if 'good' in link.get_attribute('class'):
            return True

        # Use ActionChains to move to the element and hover over it
        ActionChains(driver).move_to_element(link).perform()

        # Now retrieve the aria-describedby attribute
        aria_describedby_id = link.get_attribute('aria-describedby')

        # If the aria-describedby attribute points to an element's ID, you can then find this element
        if aria_describedby_id:
            try:
                requirements_info = driver.find_element(By.ID, aria_describedby_id).text.split('\n')
                resource_requirements = [int(item) for item in requirements_info if item.isdigit()]
            except NoSuchElementException:
                continue

            print(resource_requirements)
            if not resource_requirements:
                continue

            can_afford = True
            for i in range(4): # only need resource counts, given in order of: wood, clay, iron, wheat
                resources_available = driver.find_element(By.XPATH, f'//*[@id="l{i + 1}"]') # IDs go from l1 - l4
                resources_available = int(resources_available.text.replace(',', ''))

                if resources_available < resource_requirements[i]:
                    can_afford = False

            # if we have enough resources of all types for current field, we can afford it
            if can_afford:
                body = driver.find_element(By.TAG_NAME, 'body')
                ActionChains(driver).move_to_element(body).perform() # stop hovering
                time.sleep(0.2)
                return True

    # if we were unable to find a field we could afford
    body = driver.find_element(By.TAG_NAME, 'body')
    ActionChains(driver).move_to_element(body).perform() # stop hovering
    time.sleep(0.2)
    return False
'''


def find_lowest_level_field_of_type(driver, gid):
    navigate_to_resource_fields(driver)

    resource_field_container = WebDriverWait(driver, 12).until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, '#resourceFieldContainer'))
    )
    resource_field_links = resource_field_container.find_elements(By.TAG_NAME, 'a')
    num_res_fields = len(resource_field_links)

    target_fields = []
    for i in range(2, num_res_fields + 1): # start at 2 to skip village center
        link = driver.find_element(By.XPATH, f'//*[@id="resourceFieldContainer"]/a[{i}]')
        if 'good' in link.get_attribute('class') and f'gid{gid}' in link.get_attribute('class'):
            target_fields.append(link)

    lowest_level_field = None
    lowest_level = 999

    for field in target_fields:
        if 'underConstruction' in field.get_attribute('class'):
            continue

        curr_level = int(field.find_element(By.XPATH, './div').text.strip() or 0)
        if curr_level < lowest_level:
            lowest_level_field = field
            lowest_level = curr_level

    return lowest_level_field


def attempt_to_upgrade_lowest_level_field(drivers_info, driver, scheduler=None, retry_attempts=1):
    navigate_to_resource_fields(driver)

    try: # check if building queue is full before attempting field upgrade, skip if queue full
        building_list = driver.find_element(By.CSS_SELECTOR, '.buildingList')
        buildings_being_built = building_list.find_elements(By.TAG_NAME, 'li')

        if (not drivers_info[driver]['Gold Club'] and len(buildings_being_built) >= 1) or \
           (    drivers_info[driver]['Gold Club'] and len(buildings_being_built) >= 2):
            logs[f'{drivers_info[driver]["Username"]}_{RESOURCE_FIELDS}'] = \
                "Unable to upgrade resource field! Building queue full, skipping upgrade attempt."
            
            if scheduler:
                scheduler.add_job(attempt_to_upgrade_lowest_level_field, 'interval',
                                  seconds=calc_interval_for_server_speed(270, server_speed=SERVER_SPEED),
                                  id=f'{drivers_info[driver]["Username"]}_{RESOURCE_FIELDS}',
                                  args=[drivers_info, driver, scheduler], replace_existing=True)

            return

    except NoSuchElementException:
        logs[f'{drivers_info[driver]["Username"]}_{RESOURCE_FIELDS}'] = "Nothing currently being built, proceeding."

    lowest_level_field = None
    lowest_level = 999
    gid_types = [WOOD, CLAY, IRON, WHEAT]  # List of gid types you want to check

    for gid in gid_types:
        field = find_lowest_level_field_of_type(driver, gid)
        if field:
            curr_level = int(field.find_element(By.XPATH, './div').text.strip() or 0)
            if curr_level < lowest_level:
                lowest_level_field = field
                lowest_level = curr_level

    if lowest_level_field:
        lowest_level_field.click()

        button_upgrade = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, '.textButtonV1.green.build'))
        )
        button_upgrade.click()

        logs[f'{drivers_info[driver]["Username"]}_{RESOURCE_FIELDS}'] = "Began upgrading resource field."
    else: # if no available fields to upgrade, none can be afforded
        #if not can_afford_resource_field_upgrade(driver):
        if retry_attempts > 0:
            #logs[f'{drivers_info[driver]["Username"]}_{RESOURCE_FIELDS}'] = "Not enough resources to upgrade!"
            logs[f'{drivers_info[driver]["Username"]}_{RESOURCE_FIELDS}'] = "Not enough resources to upgrade! Refilling resources and trying again."
            get_resources_from_hero(driver)
            attempt_to_upgrade_lowest_level_field(drivers_info, driver, scheduler, retry_attempts - 1)

    if scheduler:
        scheduler.add_job(attempt_to_upgrade_lowest_level_field, 'interval',
                          seconds=calc_interval_for_server_speed(270, server_speed=SERVER_SPEED),
                          id=f'{drivers_info[driver]["Username"]}_{RESOURCE_FIELDS}',
                          args=[drivers_info, driver, scheduler], replace_existing=True)

In [12]:
''' war jobs '''


def convert_timer_time_to_seconds(time_str): # in format of HH:MM:SS
    # Parse the time string into a datetime object
    time = datetime.strptime(time_str, "%H:%M:%S")

    # Extract hours, minutes, and seconds and convert to total seconds
    seconds = time.hour * 3600 + time.minute * 60 + time.second
    return seconds



def incoming_attack(drivers_info, driver, scheduler):
    navigate_to_resource_fields(driver)

    try:
        troop_movements_container = driver.find_element(By.ID, 'movements')
        troop_movements_rows = troop_movements_container.find_elements(By.TAG_NAME, 'tr')
    except NoSuchElementException:
        logs[f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}'] = "No troop movements found."
        
        scheduler.add_job(incoming_attack, 'interval',
                          seconds=calc_interval_for_server_speed(600, server_speed=SERVER_SPEED),
                          id=f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}',
                          args=[drivers_info, driver, scheduler], replace_existing=True)
        
        return
    
    try:
        incoming_troops_row = troop_movements_container.find_element(By.XPATH, ".//th[contains(text(), 'Incoming')]")
    except NoSuchElementException:
        logs[f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}'] = "No incoming troops found."

        scheduler.add_job(incoming_attack, 'interval',
                          seconds=calc_interval_for_server_speed(600, server_speed=SERVER_SPEED),
                          id=f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}',
                          args=[drivers_info, driver, scheduler], replace_existing=True)

        return

    for troop_movement in troop_movements_rows:
        try:
            th = troop_movement.find_element(By.TAG_NAME, 'th')
            if 'Outgoing' in th.text: # if we've hit the Outgoing Troops row/section, we're done
                break
            else: # else this is the Incoming Troops row, skip it
                continue
        except NoSuchElementException: # if no th element found, this is a troop movement
            pass

        try:
            img = troop_movement.find_element(By.XPATH, './td/a/img')
        except NoSuchElementException:
            logs[f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}'] = "No image found."
            continue

        if 'att1' in img.get_attribute('class'):
            timer = troop_movement.find_element(By.CSS_SELECTOR, '.timer')
            seconds_till_attack = int(timer.get_attribute('value'))

            logs[f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}'] = "Incoming attack found! Setting job to spend all resources before attack lands."
            
            scheduler.add_job(spend_all_resources_on_troop_production, 'date',
                              run_date=datetime.now() + timedelta(seconds=seconds_till_attack - 120/SERVER_SPEED),
                              id=f'{drivers_info[driver]["Username"]}_{SPEND_ALL_RESOURCES}',
                              args=[drivers_info, driver], replace_existing=True)
            
            scheduler.add_job(incoming_attack, 'interval',
                              seconds=calc_interval_for_server_speed(600, server_speed=SERVER_SPEED),
                              id=f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}',
                              args=[drivers_info, driver, scheduler], replace_existing=True)

            return True, seconds_till_attack

    logs[f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}'] = "No incoming attacks!"

    scheduler.add_job(incoming_attack, 'interval',
                      seconds=calc_interval_for_server_speed(600, server_speed=SERVER_SPEED),
                      id=f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}',
                      args=[drivers_info, driver, scheduler], replace_existing=True)

    return False, -1


def outgoing_attack(drivers_info, driver, scheduler):
    navigate_to_resource_fields(driver)

    try:
        troop_movements_container = driver.find_element(By.ID, 'movements')
        troop_movements_rows = troop_movements_container.find_elements(By.TAG_NAME, 'tr')
    except NoSuchElementException:
        print("No troop movements found.")
        return
    
    try:
        outgoing_troops_row = troop_movements_container.find_element(By.XPATH, ".//th[contains(text(), 'Outgoing')]")
    except NoSuchElementException:
        print("No outgoing troops found.")
        return

    outgoing_troops_start_index = next((i for i, element in enumerate(troop_movements_rows) if 'test' in element.text), None)
    for troop_movement in troop_movements_rows[outgoing_troops_start_index + 1]:    
        try:
            img = troop_movement.find_element(By.TAG_NAME, 'img')
        except NoSuchElementException:
            continue

        if 'def2' in img.get_attribute('class'):
            timer = troop_movement.find_element(By.CSS_SELECTOR, '.timer')
            seconds_till_attack = int(timer.get_attribute('value'))

            print(f"Reinforcements incoming in {seconds_till_attack} seconds.")

            return True, seconds_till_attack

    print("No reinforcements coming.")
    return False, -1


def spend_all_resources_on_troop_production(drivers_info, driver):
    train_troops(drivers_info, driver, drivers_info[driver]['Troop Building'], drivers_info[driver]['Troop Name'], True)


def train_troops(drivers_info, driver, building, troop_name, incoming_attack_imminent=False, scheduler=None):
    enter_building(driver, building)
    if troop_name is None or pd.isna(troop_name):
        logs[f'{drivers_info[driver]["Username"]}_{TRAIN_TROOPS}'] = "Troop Name is undefined!"
        return

    try:
        link_troop_name = driver.find_element(By.XPATH, f"//a[contains(text(), '{troop_name}')]")
        target_troop_container = link_troop_name.find_element(By.XPATH, '../../..')
    except NoSuchElementException:
        logs[f'{drivers_info[driver]["Username"]}_{TRAIN_TROOPS}'] = \
            f"Troop {troop_name} not found. Please check troop name and spelling."
        return

    if incoming_attack_imminent:
        button_exchange_resources = WebDriverWait(driver, 7).until(EC.presence_of_element_located(
            (By.XPATH, '//button[contains(text(), "Exchange")]'))
        )
        button_exchange_resources.click()

        button_distribute_remaining_resources = WebDriverWait(driver, 7).until(EC.presence_of_element_located(
            (By.XPATH, '//button[contains(@value, "Distribute")]'))
        )
        button_distribute_remaining_resources.click()

        button_redeem = WebDriverWait(driver, 7).until(EC.presence_of_element_located(
            (By.ID, 'npc_market_button'))
        )
        #EC.element_to_be_clickable not functioning consistently, manually wait to ensure button is clickable
        time.sleep(5)
        button_redeem.click()

        time.sleep(1) # allow page time to refresh after resource distribution

        try: #re-init troop container to avoid stale references
            link_troop_name = driver.find_element(By.XPATH, f"//a[contains(text(), '{troop_name}')]")
            target_troop_container = link_troop_name.find_element(By.XPATH, '../../..')
        except NoSuchElementException:
            logs[f'{drivers_info[driver]["Username"]}_{TRAIN_TROOPS}'] = \
                f"Troop {troop_name} not found. Please check troop name and spelling."

    try: # div container changes when trainable troops is 0
        input_num_troops_to_train = target_troop_container.find_element(By.XPATH, './div[2]/div[4]/input')
    except NoSuchElementException:
        logs[f'{drivers_info[driver]["Username"]}_{TRAIN_TROOPS}'] = "Cannot afford to train any troops!"
        if scheduler:
            scheduler.add_job(train_troops, 'interval',
                              seconds=calc_interval_for_server_speed(1800, server_speed=SERVER_SPEED),
                              id=f'{drivers_info[driver]["Username"]}_{TRAIN_TROOPS}', args=[drivers_info, driver, building, troop_name, False, scheduler],
                              replace_existing=True)
        return
    
    max_trainable = target_troop_container.find_element(By.XPATH, "./div[2]/div[4]/a").text
    max_trainable = int(max_trainable)

    input_num_troops_to_train.clear()
    input_num_troops_to_train.send_keys(max_trainable)

    button_start_training = driver.find_element(By.ID, 's1')
    button_start_training.click()

    logs[f'{drivers_info[driver]["Username"]}_{TRAIN_TROOPS}'] = f"Successfully began training {max_trainable} {troop_name}."

    if scheduler:
        scheduler.add_job(train_troops, 'interval',
                          seconds=calc_interval_for_server_speed(1800, server_speed=SERVER_SPEED),
                          id=f'{drivers_info[driver]["Username"]}_{TRAIN_TROOPS}', args=[drivers_info, driver, building, troop_name, False, scheduler],
                          replace_existing=True)


def check_total_troop_counts():
    pass


def activate_farm_list_raids_for(list_id, driver, distance_limit=float('inf'), ignore_curr_state=False):
    try: # noob protection will be first div in "rallyPointFarmList" until shield wears off
        list_element_xpath = f'//*[@id="rallyPointFarmList"]/div[1]/div[{list_id + 1}]'
        list_element = WebDriverWait(driver, 9).until(EC.presence_of_element_located((By.XPATH, list_element_xpath)))
    except TimeoutException: # use second div if first is noob protection info.
        list_element_xpath = f'//*[@id="rallyPointFarmList"]/div[2]/div[{list_id + 1}]'
        list_element = WebDriverWait(driver, 9).until(EC.presence_of_element_located((By.XPATH, list_element_xpath)))

    name = list_element.find_element(By.XPATH, './div/div[1]/div[2]/div[1]').text

    table = list_element.find_element(By.XPATH, './div/div[2]/table')
    table_body = table.find_element(By.XPATH, './tbody')
    table_footer = table.find_element(By.XPATH, './tfoot')
    table_rows = table_body.find_elements(By.TAG_NAME, 'tr')

    boxes_to_click_ids = []
    for row_index in range(1, len(table_rows)): # purposefully removed last index to skip "Add target" row
        row_xpath = f'{list_element_xpath}/div/div[2]/table/tbody/tr[{row_index}]'
        try:
            row = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, row_xpath)))
        except TimeoutException:
            break

        checkbox_xpath = None
        try:
            checkbox = row.find_element(By.XPATH, './td[1]/label/input')
            checkbox_xpath = row_xpath + '/td[1]/label/input'
            driver.execute_script(JS_SCROLL_INTO_VIEW, checkbox)
        except NoSuchElementException:
            continue

        curr_state_container = row.find_element(By.CSS_SELECTOR, '.state')
        currently_attacking = False
        try:
            curr_state = curr_state_container.find_element(By.TAG_NAME, 'i')
            if 'attack_small' in curr_state.get_attribute('class'):
                currently_attacking = True
        except NoSuchElementException:
            currently_attacking = False

        dist_container = row.find_element(By.CSS_SELECTOR, '.distance')
        dist = float(dist_container.find_element(By.XPATH, './span').text)
        if dist > distance_limit:
            break

        num_troops_container = row.find_element(By.CSS_SELECTOR, '.troops')
        num_troops = int(num_troops_container.find_element(By.XPATH, './div[1]/span[1]/span[1]').text)

        try:
            filthy_nominator = driver.find_element(By.XPATH, f'{list_element_xpath}/div/div[2]/table/tfoot/tr[1]/td[2]/div/div/span/span/span[1]').text
            filthy_denominator = driver.find_element(By.XPATH, f'{list_element_xpath}/div/div[2]/table/tfoot/tr[1]/td[2]/div/div/span/span/span[1]').text
            nominator   = int(re.sub(r'[^\d]', '', filthy_nominator))
            denominator = int(re.sub(r'[^\d]', '', filthy_denominator))
        except NoSuchElementException:
            nominator = denominator = None
        
        target = row.find_element(By.CSS_SELECTOR, '.target')
        target_link = target.find_element(By.XPATH, './a')

        last_raid_state_container = row.find_element(By.CSS_SELECTOR, '.lastRaid')
        prev_attack_suffered_losses = False
        try:
            last_raid_state = last_raid_state_container.find_element(By.TAG_NAME, 'i')
            if 'attack_won_withoutLosses_small' not in last_raid_state.get_attribute('class'):
                prev_attack_suffered_losses = True
        except NoSuchElementException:
            prev_attack_suffered_losses = False

        if (not currently_attacking or ignore_curr_state) and \
        (not prev_attack_suffered_losses or (name.lower() == 'oases' and not oases_has_troops(driver, target_link))) and \
        ((not nominator and not denominator) or nominator + num_troops <= denominator): # TODO: THIS CHECK IS FAILING
            boxes_to_click_ids.append(checkbox_xpath)
        
    for id in boxes_to_click_ids:
        try:
            checkbox = WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.XPATH, id)))
            driver.execute_script(JS_SCROLL_INTO_VIEW, checkbox)
        except NoSuchElementException:
            continue

        checkbox.click()

    # start raids
    if boxes_to_click_ids:
        list_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, list_element_xpath)))
        button_start_raids = list_element.find_element(By.XPATH, './div/div[1]/button')
        driver.execute_script(JS_SCROLL_INTO_VIEW, button_start_raids)
        button_start_raids.click()


def has_gold_club_membership(drivers_info, driver, scheduler=None):
    enter_building(driver, 'Rally Point')
    
    try:
        button_farm_list = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
            (By.XPATH, '//a[contains(@href, "/build.php?id=39&gid=16&tt=99")]'))
        )
        button_farm_list.click()

        if scheduler and scheduler.get_job(f'{drivers_info[driver]["Username"]}_{GOLD_CLUB_CHECK}'):
            scheduler.remove_job(f'{drivers_info[driver]["Username"]}_{GOLD_CLUB_CHECK}')

        return True
    except NoSuchElementException:
        log_msg = "Please activate Travian Gold Club to gain access to farm lists."
        logs[f'{drivers_info[driver]["Username"]}_{GOLD_CLUB_CHECK}'] = log_msg
        logs[f'{drivers_info[driver]["Username"]}_{RAIDS}'] = log_msg
        
        if scheduler:
            scheduler.add_job(has_gold_club_membership, 'interval',
                              seconds=calc_interval_for_server_speed(10800, server_speed=SERVER_SPEED),
                              id=f'{drivers_info[driver]["Username"]}_{GOLD_CLUB_CHECK}',
                              args=[drivers_info, driver, scheduler], replace_existing=True)
        
        return False


def oases_has_troops(driver, target):
    target.click()

    try:
        first_troop_row_text = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="troop_info"]/tbody/tr[1]/td'))
        ).text
        has_troops = (first_troop_row_text != 'none')
    except TimeoutException:
        has_troops = True

    driver.back()
    return has_troops

  

def send_troops_to_farm(drivers_info, driver, scheduler=None):
    if 'build.php?id=39&gid=16&tt=99' not in driver.current_url:
        enter_building(driver, 'Rally Point')

        try:
            button_farm_list = WebDriverWait(driver, 9).until(EC.presence_of_element_located(
                (By.XPATH, '//a[contains(@href, "/build.php?id=39&gid=16&tt=99")]'))
            )
            button_farm_list.click()
        except TimeoutException:
            logs[f'{drivers_info[driver]["Username"]}_{RAIDS}'] = "Please activate Travian Gold Club to gain access to farm lists."
            if scheduler:
                scheduler.pause_job(job_id=f'{drivers_info[driver]["Username"]}_{RAIDS}')
            return


    try:
        farm_lists_container = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, '.villageWrapper'))
        )
    except TimeoutException:
        logs[f'{drivers_info[driver]["Username"]}_{RAIDS}'] = "Please create a farm list to begin raiding!"
        if scheduler:
            scheduler.pause_job(job_id=f'{drivers_info[driver]["Username"]}_{RAIDS}')
        return

    farm_lists = farm_lists_container.find_elements(By.CSS_SELECTOR, '.dropContainer')
    for farm_list in farm_lists:
        name = farm_list.find_element(By.XPATH, './div/div[1]/div[2]/div[1]').text

        farm_list_index = driver.execute_script("return Array.prototype.indexOf.call(arguments[0].parentNode.children, arguments[0]);", farm_list)
        activate_farm_list_raids_for(farm_list_index, driver, distance_limit=7, ignore_curr_state=False)
        
        logs[f'{drivers_info[driver]["Username"]}_{RAIDS}'] = f'Finished raid logic for {name}'


    logs[f'{drivers_info[driver]["Username"]}_{RAIDS}'] = "Finished raid attempt."

    if scheduler:
        scheduler.add_job(send_troops_to_farm, 'interval',
                          seconds=calc_interval_for_server_speed(900, server_speed=SERVER_SPEED),
                          id=f'{drivers_info[driver]["Username"]}_{RAIDS}',
                          args=[drivers_info, driver, scheduler], replace_existing=True)

In [13]:
''' mission/daily rewards collection jobs '''

def collect_mission_resources(drivers_info, driver, scheduler=None):
    # button should exist on all pages, if not we've encountered an error. Refresh page and try again
    try:
        button_mentor = driver.find_element(By.ID, 'questmasterButton')
    except NoSuchElementException:
        driver.refresh()
        button_mentor = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
            (By.ID, 'questmasterButton'))
        )

    try:
        speech_bubble = button_mentor.find_element(By.XPATH, './div')
    except NoSuchElementException:
        logs[f'{drivers_info[driver]["Username"]}_{COLLECT_MISSION_RESOURCES}'] = "No mission resources to collect!"
        if scheduler:
            scheduler.add_job(collect_mission_resources, 'interval',
                              seconds=calc_interval_for_server_speed(3600, server_speed=SERVER_SPEED),
                              id=f'{drivers_info[driver]["Username"]}_{COLLECT_MISSION_RESOURCES}', 
                              args=[drivers_info, driver, scheduler], replace_existing=True)
        return

    button_mentor.click()

    task_overview = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.taskOverview')))    

    task_list = task_overview.find_elements(By.XPATH, './div')
    for task in task_list:
        if 'achieved' in task.get_attribute('class'):
            button_collect = task.find_element(By.TAG_NAME, 'button')
            button_collect.click()
    

    logs[f'{drivers_info[driver]["Username"]}_{COLLECT_MISSION_RESOURCES}'] = "Successfully collected mission resources."
    if scheduler:
        scheduler.add_job(collect_mission_resources, 'interval',
                          seconds=calc_interval_for_server_speed(3600, server_speed=SERVER_SPEED),
                          id=f'{drivers_info[driver]["Username"]}_{COLLECT_MISSION_RESOURCES}',
                          args=[drivers_info, driver, scheduler], replace_existing=True)


def collect_daily_quests_rewards(drivers_info, driver, scheduler=None):
    button_daily_quests = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@id="navigation"]/a[7]'))
    )

    try:
        indicator = button_daily_quests.find_element(By.XPATH, './div')
    except NoSuchElementException:
        logs[f'{drivers_info[driver]["Username"]}_{COLLECT_DAILY_QUEST_REWARDS}'] = "No rewards to collect!"
        if scheduler:
            scheduler.add_job(collect_daily_quests_rewards, 'interval',
                              seconds=calc_interval_for_server_speed(10800, server_speed=SERVER_SPEED),
                              id=f'{drivers_info[driver]["Username"]}_{COLLECT_DAILY_QUEST_REWARDS}',
                              args=[drivers_info, driver, scheduler], replace_existing=True)
        return

    button_daily_quests.click()

    achievement_reward_list = WebDriverWait(driver, 7).until(EC.presence_of_element_located(
        (By.ID, 'achievementRewardList'))
    )
    reward_container_elements = achievement_reward_list.find_elements(By.TAG_NAME, 'a')
    num_rewards = len(reward_container_elements)
    for i in range(1, num_rewards + 1): # first reward button starts at 3rd div
        container = driver.find_element(By.CSS_SELECTOR, f'#achievementRewardList > a:nth-of-type({i})')
        driver.execute_script(JS_CLICK, container) # link has size of zero, must use js to click


        button_collect_reward = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.textButtonV1.green.questButtonGainReward')))
        button_collect_reward.click()

    logs[f'{drivers_info[driver]["Username"]}_{COLLECT_DAILY_QUEST_REWARDS}'] = "Successfully collected mission resources."
    
    if scheduler:
        scheduler.add_job(collect_daily_quests_rewards, 'interval',
                          seconds=calc_interval_for_server_speed(10800, server_speed=SERVER_SPEED),
                          id=f'{drivers_info[driver]["Username"]}_{COLLECT_DAILY_QUEST_REWARDS}',
                          args=[drivers_info, driver, scheduler], replace_existing=True)

In [14]:
''' hero jobs '''

def attempt_to_start_adventure(drivers_info, driver, scheduler=None):
    # hero button should exist on all pages, if not we've encountered an error. Refresh page and try again
    try:
        button_hero_status = driver.find_element(By.XPATH, '//*[@id="topBarHero"]/div/a/i')
    except NoSuchElementException:
        driver.refresh()
        button_hero_status = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="topBarHero"]/div/a/i')))


    # dynamic ID, using href to reference
    button_adventures = driver.find_element(By.XPATH, "//a[@href='/hero/adventures']")
    try:
        num_adventures = button_adventures.find_element(By.XPATH, "./div")
    except NoSuchElementException:
        num_adventures = None

    if num_adventures and 'heroHome' in button_hero_status.get_attribute('class') and int(num_adventures.text) > 0:
        button_adventures.click()

        button_start_first_adventure = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="heroAdventure"]/table/tbody/tr[1]/td[5]/button'))
        )
        button_start_first_adventure.click()

        button_continue = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="heroAdventure"]/div/button')))
        button_continue.click()

        logs[f'{drivers_info[driver]["Username"]}_{ADVENTURES}'] = "Successfully sent out hero on adventure!"
    elif num_adventures is None:
        logs[f'{drivers_info[driver]["Username"]}_{ADVENTURES}'] = "No adventures found."
    elif 'heroRunning' in button_hero_status.get_attribute('class'):
        logs[f'{drivers_info[driver]["Username"]}_{ADVENTURES}'] = "Hero is already on an adventure!"
    else:
        logs[f'{drivers_info[driver]["Username"]}_{ADVENTURES}'] = "Error attempting to start adventure."
    
    if scheduler:
        scheduler.add_job(attempt_to_start_adventure, 'interval',
                          seconds=calc_interval_for_server_speed(10800, server_speed=SERVER_SPEED),
                          id=f'{drivers_info[driver]["Username"]}_{ADVENTURES}', args=[drivers_info, driver, scheduler],
                          replace_existing=True)


# possible options are 'resourceProduction', 'fightingStrength', 'offBonus', 'defBonus
def upgrade_hero(drivers_info, driver, scheduler=None, attribute_to_upgrade='resourceProduction'):
    level_up_icon = WebDriverWait(driver, 7).until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, '#topBarHero > i'))
    )
    
    if 'show' not in level_up_icon.get_attribute('class'):
        logs[f'{drivers_info[driver]["Username"]}_{HERO_UPGRADE}'] = "Hero has no points to spend!"
        if scheduler:
            #scheduler.pause_job(job_id=f'{drivers_info[driver]["Username"]}_{HERO_UPGRADE}')
            scheduler.add_job(upgrade_hero, 'interval',
                              seconds=calc_interval_for_server_speed(10800, server_speed=SERVER_SPEED),
                              id=f'{drivers_info[driver]["Username"]}_{HERO_UPGRADE}',
                              args=[drivers_info, driver, scheduler],
                              replace_existing=True)
        return
    
    navigate_to_hero_inventory(driver)

    button_attributes = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
        (By.XPATH, '//*[@id="heroV2"]/div[1]/div[1]/div/div[2]'))
    )
    button_attributes.click()

    input_points = WebDriverWait(driver, 5).until(EC.presence_of_element_located(
        (By.NAME, attribute_to_upgrade))
    )
    curr_num_points = int(input_points.get_attribute('value'))
    new_num_points = curr_num_points + 4 # hero always gets 4 points to spend after leveling up
    
    input_points.clear()
    input_points.send_keys(str(new_num_points))

    button_save_changes = driver.find_element(By.ID, 'savePoints')
    time.sleep(2) # button click doesn't register if we move too fast
    button_save_changes.click()

    logs[f'{drivers_info[driver]["Username"]}_{HERO_UPGRADE}'] = f"Successfully upgraded hero and spent points on {attribute_to_upgrade}."

    if scheduler:
        scheduler.add_job(upgrade_hero, 'interval',
                          seconds=calc_interval_for_server_speed(10800, server_speed=SERVER_SPEED),
                          id=f'{drivers_info[driver]["Username"]}_{HERO_UPGRADE}',
                          args=[drivers_info, driver, scheduler],
                          replace_existing=True)

In [15]:
# TODO: Update table generation to handle multiple drivers
def generate_job_scheduler_table_from(drivers_info, scheduler):
    layout = Layout()

    now = datetime.now(timezone.utc)

    # Sort the jobs by job.id and then by countdown
    jobs = scheduler.get_jobs()
    sorted_jobs = sorted(jobs, key=lambda job: (
        job.id,
        999 if job.next_run_time is None else (job.next_run_time - now).total_seconds()
    ))

    for driver in drivers_info.keys():
        drivers_info[driver]['Table'] = Table(title=f"{drivers_info[driver]['Username']}'s Scheduled Jobs",
                                              box=box.DOUBLE, safe_box=False)
        drivers_info[driver]['Table'].width = 100

    layout = Layout()
    layout.split_column(*[drivers_info[driver]['Table'] for driver in drivers_info.keys()])

    for driver in drivers_info.keys():
        drivers_info[driver]['Table'].add_column("Job ID", style="cyan", no_wrap=True)
        drivers_info[driver]['Table'].add_column("Next Run At", style="magenta")
        drivers_info[driver]['Table'].add_column("Countdown", style="green")
        drivers_info[driver]['Table'].add_column("Log", style="blue")

        for job in sorted_jobs:
            if drivers_info[driver]['Username'] in job.id:
                next_run = job.next_run_time
                countdown = 999 if next_run is None else (next_run - now).total_seconds()
                drivers_info[driver]['Table'].add_row(job.id, str(next_run), str(int(countdown)), logs.get(job.id, ''))
        
    return layout

In [16]:
''' TESTING BLOCK

driver = init_webdriver()
drivers_info = {
    driver: {
        'Username': 'daniel.doza',
        'Type': 'Sourcer',
        'Troop Building': 'Barracks',
        'Troop Name': 'Clubswinger',
    }
}

attempt_login('https://ts2.x1.international.travian.com', drivers_info[driver]['Username'], 'Tomorrow2040!', driver)

with managed_scheduler() as scheduler:
    # enter function to test here
    get_resources_from_hero(driver) 

    with Live(generate_job_scheduler_table_from(drivers_info, scheduler), refresh_per_second=1, console=console, vertical_overflow='visible', screen=True) as live:
        try:
            while True:
                live.update(generate_job_scheduler_table_from(drivers_info, scheduler), refresh=True)

                time.sleep(1)
        except (KeyboardInterrupt, SystemExit):
            pass

driver.quit()
'''



" TESTING BLOCK\n\ndriver = init_webdriver()\ndrivers_info = {\n    driver: {\n        'Username': 'daniel.doza',\n        'Type': 'Sourcer',\n        'Troop Building': 'Barracks',\n        'Troop Name': 'Clubswinger',\n    }\n}\n\nattempt_login('https://ts2.x1.international.travian.com', drivers_info[driver]['Username'], 'Tomorrow2040!', driver)\n\nwith managed_scheduler() as scheduler:\n    # enter function to test here\n    get_resources_from_hero(driver) \n\n    with Live(generate_job_scheduler_table_from(drivers_info, scheduler), refresh_per_second=1, console=console, vertical_overflow='visible', screen=True) as live:\n        try:\n            while True:\n                live.update(generate_job_scheduler_table_from(drivers_info, scheduler), refresh=True)\n\n                time.sleep(1)\n        except (KeyboardInterrupt, SystemExit):\n            pass\n\ndriver.quit()\n"

In [17]:
''' init. and login to server (single account)

input_port = input("Port:")

try:
    with managed_scheduler(job_defaults={'misfire_grace_time': 60}) as scheduler, managed_webdriver(input_port) as driver:
        drivers_info = {
            driver: {
                'Username': 'dux',
                'Type': 'Leader',
                'Troop Building': 'Barracks',
                'Troop Name': 'Legionnaire',
            }
        }
        input_password = getpass("Password:")

        if drivers_info[driver]['Username'] and drivers_info[driver]['Type'] and input_password:
            attempt_login(input_site, drivers_info[driver]['Username'], input_password, driver)
            drivers_info[driver]['Gold Club'] = has_gold_club_membership(drivers_info, driver, scheduler=None)

            # !!! STILL IN TESTING, will fail without restarting once building queue is full or resources depleted
            #if not wall_built(driver):
            #    run_missions_and_disable_contextual_helpers(driver)

            scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=7,
                              id=f'{drivers_info[driver]["Username"]}_{ADVENTURES}', args=[drivers_info, driver, scheduler])
            scheduler.add_job(collect_mission_resources, 'interval', seconds=14,
                              id=f'{drivers_info[driver]["Username"]}_{COLLECT_MISSION_RESOURCES}', args=[drivers_info, driver, scheduler])
            scheduler.add_job(upgrade_hero, 'interval', seconds=21,
                              id=f'{drivers_info[driver]["Username"]}_{HERO_UPGRADE}', args=[drivers_info, driver, scheduler])
            scheduler.add_job(collect_daily_quests_rewards, 'interval', seconds=32,
                              id=f'{drivers_info[driver]["Username"]}_{COLLECT_DAILY_QUEST_REWARDS}', args=[drivers_info, driver, scheduler])
            scheduler.add_job(incoming_attack, 'interval', seconds=32,
                              id=f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}', args=[drivers_info, driver, scheduler])
            if drivers_info[driver]['Gold Club']:
                scheduler.add_job(send_troops_to_farm, 'interval', seconds=42,
                                  id=f'{drivers_info[driver]["Username"]}_{RAIDS}', args=[drivers_info, driver, scheduler])
            if drivers_info[driver]['Type'].lower() == 'enforcer':
                scheduler.add_job(train_troops, 'interval', seconds=45,
                                  id=f'{drivers_info[driver]["Username"]}_{TRAIN_TROOPS}',
                                  args=[drivers_info, driver,
                                        drivers_info[driver]['Troop Building'], drivers_info[driver]['Troop Name'],
                                        False, scheduler])            
            else:
                scheduler.add_job(attempt_to_upgrade_lowest_level_field, 'interval', seconds=120,
                                id=f'{drivers_info[driver]["Username"]}_{RESOURCE_FIELDS}', args=[drivers_info, driver, scheduler])
            
            # for any uncaught server errors, refresh page every hour to ensure continued job executions
            scheduler.add_job(refresh_page, 'interval', seconds=600,
                                id=f"{drivers_info[driver]['Username']}_{REFRESH}", args=[drivers_info, driver, scheduler])

            with Live(generate_job_scheduler_table_from(drivers_info, scheduler), refresh_per_second=1, console=console) as live:
                try:
                    while True:
                        live.update(generate_job_scheduler_table_from(drivers_info, scheduler))
                        time.sleep(1)
                except (KeyboardInterrupt, SystemExit):
                    pass
except Exception as e:
    print(f"Error: {e}")
'''

' init. and login to server (single account)\n\ninput_port = input("Port:")\n\ntry:\n    with managed_scheduler(job_defaults={\'misfire_grace_time\': 60}) as scheduler, managed_webdriver(input_port) as driver:\n        drivers_info = {\n            driver: {\n                \'Username\': \'dux\',\n                \'Type\': \'Leader\',\n                \'Troop Building\': \'Barracks\',\n                \'Troop Name\': \'Legionnaire\',\n            }\n        }\n        input_password = getpass("Password:")\n\n        if drivers_info[driver][\'Username\'] and drivers_info[driver][\'Type\'] and input_password:\n            attempt_login(input_site, drivers_info[driver][\'Username\'], input_password, driver)\n            drivers_info[driver][\'Gold Club\'] = has_gold_club_membership(drivers_info, driver, scheduler=None)\n\n            # !!! STILL IN TESTING, will fail without restarting once building queue is full or resources depleted\n            #if not wall_built(driver):\n           

In [18]:
''' init. and login to server (multiple accounts) '''

drivers_info = {}
user_info = {}

user_info = pd.read_excel('~/Dropbox/TravianAccounts.xlsx')
for index, row in user_info.iterrows():
    driver = init_webdriver(int(row['Port']) if not pd.isna(row['Port']) else None)
    attempt_login(input_site, row['Username'], row['Password'], driver)

    # !!! STILL IN TESTING, will fail without restarting once building queue is full
    #if not wall_built(driver):
    #    run_missions_and_disable_contextual_helpers(driver)

    drivers_info[driver] = row


'''
proxy_ports = [[None, 'enforcer'], [24000, 'leader'], [24001, 'enforcer'], [24002, 'enforcer'], [24003, 'sourcer'], [24004, 'defender']]
#for i in range(num_accounts): # get accounts info
#    proxy_port = 24000 + i
for proxy_port in proxy_ports:
    user_info[proxy_port[0]] = {}
    user_info[proxy_port[0]]['username'] = input(f"Username for proxy port {proxy_port[0]}:")
    user_info[proxy_port[0]]['password'] = getpass(f"Password for proxy port {proxy_port[0]}:")


#for i in range(num_accounts): # initialize web drivers
#    proxy_port = 24000 + i
for proxy_port in proxy_ports:
    driver = init_webdriver(proxy_port[0])
    attempt_login(input_site, user_info[proxy_port[0]]['username'], driver)

    # !!! STILL IN TESTING, will fail without restarting once building queue is full
    #if not wall_built(driver):
    #    run_missions_and_disable_contextual_helpers(driver)
    
    drivers[driver] = proxy_port[1]
'''


''' begin scheduler tasks '''
with managed_scheduler(job_defaults={'misfire_grace_time': 60}) as scheduler:
    for driver in drivers_info.keys():
        drivers_info[driver]['Gold Club'] = has_gold_club_membership(drivers_info, driver, scheduler=None)

        scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=7,
                          id=f'{drivers_info[driver]["Username"]}_{ADVENTURES}', args=[drivers_info, driver, scheduler])
        scheduler.add_job(collect_mission_resources, 'interval', seconds=14,
                          id=f'{drivers_info[driver]["Username"]}_{COLLECT_MISSION_RESOURCES}', args=[drivers_info, driver, scheduler])
        scheduler.add_job(upgrade_hero, 'interval', seconds=21,
                          id=f'{drivers_info[driver]["Username"]}_{HERO_UPGRADE}', args=[drivers_info, driver, scheduler])
        scheduler.add_job(collect_daily_quests_rewards, 'interval', seconds=28,
                          id=f'{drivers_info[driver]["Username"]}_{COLLECT_DAILY_QUEST_REWARDS}', args=[drivers_info, driver, scheduler])
        scheduler.add_job(incoming_attack, 'interval', seconds=35,
                          id=f'{drivers_info[driver]["Username"]}_{CHECK_FOR_INCOMING_ATTACKS}', args=[drivers_info, driver, scheduler])
        if drivers_info[driver]['Upgrade Fields']:
            scheduler.add_job(attempt_to_upgrade_lowest_level_field, 'interval', seconds=42,
                              id=f'{drivers_info[driver]["Username"]}_{RESOURCE_FIELDS}', args=[drivers_info, driver, scheduler])
        if drivers_info[driver]['Type'].lower() == 'enforcer' and drivers_info[driver]['Train Troops']:
            scheduler.add_job(train_troops, 'interval', seconds=88,
                              id=f'{drivers_info[driver]["Username"]}_{TRAIN_TROOPS}',
                              args=[drivers_info, driver,
                                    drivers_info[driver]['Troop Building'], drivers_info[driver]['Troop Name'],
                                    False, scheduler])
        if drivers_info[driver]['Gold Club'] and drivers_info[driver]['Raid']:
            scheduler.add_job(send_troops_to_farm, 'interval', seconds=120,
                              id=f'{drivers_info[driver]["Username"]}_{RAIDS}', args=[drivers_info, driver, scheduler])

        # for any uncaught server errors, refresh page every hour to ensure continued job executions
        scheduler.add_job(refresh_page, 'interval', seconds=600,
                              id=f"{drivers_info[driver]['Username']}_{REFRESH}", args=[drivers_info, driver, scheduler])
    
    with Live(generate_job_scheduler_table_from(drivers_info, scheduler), refresh_per_second=1, console=console, vertical_overflow='visible', screen=True) as live:
        try:
            while True:
                live.update(generate_job_scheduler_table_from(drivers_info, scheduler), refresh=True)

                time.sleep(1)
        except (KeyboardInterrupt, SystemExit):
            pass

#for driver in drivers_info.keys():
#    driver.quit()

run_command_in_background('curl -X POST "http://127.0.0.1:22999/api/shutdown"')

Output()

Job "send_troops_to_farm (trigger: interval[0:05:10.162805], next run at: 2024-03-23 00:41:00 MDT)" raised an 
exception
Traceback (most recent call last):
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 202, in 
activate_farm_list_raids_for
    list_element = WebDriverWait(driver, 9).until(EC.presence_of_element_located((By.XPATH, list_element_xpath)))
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/support/wait.py"
, line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001007383e8 chromedriver + 4326376
1   chromedriver                        0x00000001007308b0 chromedriver + 4294832
2   chromedriver                        0x000000010035c088 chromedriver + 278664
3   chromedriver                        0x000000010039ea80 chromedriver + 551552
4   chromedriver                        0x00000001003d74f8 chromedriver + 783608
5   chromedriver                        0x00000001003934e4 chromedriver + 505060
6   chromedriver                        0x0000000100393f5c chromedriver + 507740
7   chromedriver                        0x00000001006fb984 chromedriver + 4077956
8   chromedriver                        0x000000010070073c chromedriver + 4097852
9   chromedriver                        0x00000001006e2528 chromedriver + 3974440
10  chromedriver                        0x0000000100701054 chromedriver + 4100180
11  chromedriver                        0x00000001006d3b18 chromedriver + 3914520
12  chromedriver                        0x000000010072165c chromedriver + 4232796
13  chromedriver                        0x00000001007217d8 chromedriver + 4233176
14  chromedriver                        0x0000000100730524 chromedriver + 4293924
15  libsystem_pthread.dylib             0x0000000183d43034 _pthread_start + 136
16  libsystem_pthread.dylib             0x0000000183d3de3c thread_start + 8


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 363, in 
send_troops_to_farm
    activate_farm_list_raids_for(farm_list_index, driver, distance_limit=7, ignore_curr_state=False)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 205, in 
activate_farm_list_raids_for
    list_element = WebDriverWait(driver, 9).until(EC.presence_of_element_located((By.XPATH, list_element_xpath)))
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/support/wait.py"
, line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001007383e8 chromedriver + 4326376
1   chromedriver                        0x00000001007308b0 chromedriver + 4294832
2   chromedriver                        0x000000010035c088 chromedriver + 278664
3   chromedriver                        0x000000010039ea80 chromedriver + 551552
4   chromedriver                        0x00000001003d74f8 chromedriver + 783608
5   chromedriver                        0x00000001003934e4 chromedriver + 505060
6   chromedriver                        0x0000000100393f5c chromedriver + 507740
7   chromedriver                        0x00000001006fb984 chromedriver + 4077956
8   chromedriver                        0x000000010070073c chromedriver + 4097852
9   chromedriver                        0x00000001006e2528 chromedriver + 3974440
10  chromedriver                        0x0000000100701054 chromedriver + 4100180
11  chromedriver   

Job "send_troops_to_farm (trigger: interval[0:04:54.361207], next run at: 2024-03-23 01:01:07 MDT)" raised an 
exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 363, in 
send_troops_to_farm
    activate_farm_list_raids_for(farm_list_index, driver, distance_limit=7, ignore_curr_state=False)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 256, in 
activate_farm_list_raids_for
    target_link = target.find_element(By.XPATH, './a')
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webelemen
t.py", line 417, in find_element
    return self._execute(Command.FIND_CHILD_ELEMENT, {"using": by, "value": value})["value"]
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webelemen
t.py", line 395, in _execute
    return self._parent.execute(command, params)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver
.py", line 347, in execute
    self.error_handler.check_response(response)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/errorhand
ler.py", line 229, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.StaleElementReferenceException: Message: stale element reference: stale element not 
found
  (Session info: chrome=123.0.6312.59); For documentation on this error, please visit: 
https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102c2c3e8 chromedriver + 4326376
1   chromedriver                        0x0000000102c248b0 chromedriver + 4294832
2   chromedriver                        0x0000000102850088 chromedriver + 278664
3   chromedriver                        0x000000010285ddb8 chromedriver + 335288
4   chromedriver                        0x0000000102855370 chromedriver + 299888
5   chromedriver                        0x0000000102853c14 chromedriver + 293908
6   chromedriver                        0x0000000102856428 chromedriver + 304168
7   chromedriver                        0x00000001028564a0 chromedriver + 304288
8   chromedriver                        0x0000000102892b7c chromedriver + 551804
9   chromedriver                        0x0000000102888dd0 chromedriver + 511440
10  chromedriver                        0x00000001028cb4f8 chromedriver + 783608
11  chromedriver                        0x00000001028874e4 chromedriver + 505060
12  chromedriver                        0x0000000102887f5c chromedriver + 507740
13  chromedriver                        0x0000000102bef984 chromedriver + 4077956
14  chromedriver                        0x0000000102bf473c chromedriver + 4097852
15  chromedriver                        0x0000000102bd6528 chromedriver + 3974440
16  chromedriver                        0x0000000102bf5054 chromedriver + 4100180
17  chromedriver                        0x0000000102bc7b18 chromedriver + 3914520
18  chromedriver                        0x0000000102c1565c chromedriver + 4232796
19  chromedriver                        0x0000000102c157d8 chromedriver + 4233176
20  chromedriver                        0x0000000102c24524 chromedriver + 4293924
21  libsystem_pthread.dylib             0x0000000183d43034 _pthread_start + 136
22  libsystem_pthread.dylib             0x0000000183d3de3c thread_start + 8

Job "send_troops_to_farm (trigger: interval[0:05:02.418606], next run at: 2024-03-23 01:16:43 MDT)" raised an 
exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 363, in 
send_troops_to_farm
    activate_farm_list_raids_for(farm_list_index, driver, distance_limit=7, ignore_curr_state=False)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 245, in 
activate_farm_list_raids_for
    num_troops = int(num_troops_container.find_element(By.XPATH, './div[1]/span[1]/span[1]').text)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webelemen
t.py", line 417, in find_element
    return self._execute(Command.FIND_CHILD_ELEMENT, {"using": by, "value": value})["value"]
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webelemen
t.py", line 395, in _execute
    return self._parent.execute(command, params)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver
.py", line 347, in execute
    self.error_handler.check_response(response)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/errorhand
ler.py", line 229, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.StaleElementReferenceException: Message: stale element reference: stale element not 
found
  (Session info: chrome=123.0.6312.59); For documentation on this error, please visit: 
https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000102c2c3e8 chromedriver + 4326376
1   chromedriver                        0x0000000102c248b0 chromedriver + 4294832
2   chromedriver                        0x0000000102850088 chromedriver + 278664
3   chromedriver                        0x000000010285ddb8 chromedriver + 335288
4   chromedriver                        0x0000000102855370 chromedriver + 299888
5   chromedriver                        0x0000000102853c14 chromedriver + 293908
6   chromedriver                        0x0000000102856428 chromedriver + 304168
7   chromedriver                        0x00000001028564a0 chromedriver + 304288
8   chromedriver                        0x0000000102892b7c chromedriver + 551804
9   chromedriver                        0x0000000102888dd0 chromedriver + 511440
10  chromedriver                        0x00000001028cb4f8 chromedriver + 783608
11  chromedriver                        0x00000001028874e4 chromedriver + 505060
12  chromedriver                        0x0000000102887f5c chromedriver + 507740
13  chromedriver                        0x0000000102bef984 chromedriver + 4077956
14  chromedriver                        0x0000000102bf473c chromedriver + 4097852
15  chromedriver                        0x0000000102bd6528 chromedriver + 3974440
16  chromedriver                        0x0000000102bf5054 chromedriver + 4100180
17  chromedriver                        0x0000000102bc7b18 chromedriver + 3914520
18  chromedriver                        0x0000000102c1565c chromedriver + 4232796
19  chromedriver                        0x0000000102c157d8 chromedriver + 4233176
20  chromedriver                        0x0000000102c24524 chromedriver + 4293924
21  libsystem_pthread.dylib             0x0000000183d43034 _pthread_start + 136
22  libsystem_pthread.dylib             0x0000000183d3de3c thread_start + 8

Job "incoming_attack (trigger: interval[0:03:22.256796], next run at: 2024-03-23 01:15:39 MDT)" raised an exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 15, in 
incoming_attack
    navigate_to_resource_fields(driver)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1192679095.py", line 34, in 
navigate_to_resource_fields
    navigate_to_page(driver, selector, url_check)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1192679095.py", line 20, in 
navigate_to_page
    button_page.click()
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webelemen
t.py", line 94, in click
    self._execute(Command.CLICK_ELEMENT)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webelemen
t.py", line 395, in _execute
    return self._parent.execute(command, params)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver
.py", line 347, in execute
    self.error_handler.check_response(response)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/errorhand
ler.py", line 229, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.StaleElementReferenceException: Message: stale element reference: stale element not 
found
  (Session info: chrome=123.0.6312.59); For documentation on this error, please visit: 
https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x000000010482c3e8 chromedriver + 4326376
1   chromedriver                        0x00000001048248b0 chromedriver + 4294832
2   chromedriver                        0x0000000104450088 chromedriver + 278664
3   chromedriver                        0x000000010445ddb8 chromedriver + 335288
4   chromedriver                        0x0000000104455370 chromedriver + 299888
5   chromedriver                        0x0000000104453c14 chromedriver + 293908
6   chromedriver                        0x0000000104456428 chromedriver + 304168
7   chromedriver                        0x00000001044564a0 chromedriver + 304288
8   chromedriver                        0x0000000104494450 chromedriver + 558160
9   chromedriver                        0x0000000104489264 chromedriver + 512612
10  chromedriver                        0x0000000104488dd0 chromedriver + 511440
11  chromedriver                        0x00000001044cb4f8 chromedriver + 783608
12  chromedriver                        0x00000001044874e4 chromedriver + 505060
13  chromedriver                        0x0000000104487f5c chromedriver + 507740
14  chromedriver                        0x00000001047ef984 chromedriver + 4077956
15  chromedriver                        0x00000001047f473c chromedriver + 4097852
16  chromedriver                        0x00000001047d6528 chromedriver + 3974440
17  chromedriver                        0x00000001047f5054 chromedriver + 4100180
18  chromedriver                        0x00000001047c7b18 chromedriver + 3914520
19  chromedriver                        0x000000010481565c chromedriver + 4232796
20  chromedriver                        0x00000001048157d8 chromedriver + 4233176
21  chromedriver                        0x0000000104824524 chromedriver + 4293924
22  libsystem_pthread.dylib             0x0000000183d43034 _pthread_start + 136
23  libsystem_pthread.dylib             0x0000000183d3de3c thread_start + 8

...

Job "incoming_attack (trigger: interval[0:03:20.638584], next run at: 2024-03-23 02:53:28 MDT)" raised an exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 15, in 
incoming_attack
    navigate_to_resource_fields(driver)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1192679095.py", line 34, in 
navigate_to_resource_fields
    navigate_to_page(driver, selector, url_check)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1192679095.py", line 20, in 
navigate_to_page
    button_page.click()
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webelemen
t.py", line 94, in click
    self._execute(Command.CLICK_ELEMENT)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webelemen
t.py", line 395, in _execute
    return self._parent.execute(command, params)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver
.py", line 347, in execute
    self.error_handler.check_response(response)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/errorhand
ler.py", line 229, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.StaleElementReferenceException: Message: stale element reference: stale element not 
found
  (Session info: chrome=123.0.6312.59); For documentation on this error, please visit: 
https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x00000001007383e8 chromedriver + 4326376
1   chromedriver                        0x00000001007308b0 chromedriver + 4294832
2   chromedriver                        0x000000010035c088 chromedriver + 278664
3   chromedriver                        0x0000000100369db8 chromedriver + 335288
4   chromedriver                        0x0000000100361370 chromedriver + 299888
5   chromedriver                        0x000000010035fc14 chromedriver + 293908
6   chromedriver                        0x0000000100362428 chromedriver + 304168
7   chromedriver                        0x00000001003624a0 chromedriver + 304288
8   chromedriver                        0x00000001003a0450 chromedriver + 558160
9   chromedriver                        0x0000000100395264 chromedriver + 512612
10  chromedriver                        0x0000000100394dd0 chromedriver + 511440
11  chromedriver                        0x00000001003d74f8 chromedriver + 783608
12  chromedriver                        0x00000001003934e4 chromedriver + 505060
13  chromedriver                        0x0000000100393f5c chromedriver + 507740
14  chromedriver                        0x00000001006fb984 chromedriver + 4077956
15  chromedriver                        0x000000010070073c chromedriver + 4097852
16  chromedriver                        0x00000001006e2528 chromedriver + 3974440
17  chromedriver                        0x0000000100701054 chromedriver + 4100180
18  chromedriver                        0x00000001006d3b18 chromedriver + 3914520
19  chromedriver                        0x000000010072165c chromedriver + 4232796
20  chromedriver                        0x00000001007217d8 chromedriver + 4233176
21  chromedriver                        0x0000000100730524 chromedriver + 4293924
22  libsystem_pthread.dylib             0x0000000183d43034 _pthread_start + 136
23  libsystem_pthread.dylib             0x0000000183d3de3c thread_start + 8

Job "send_troops_to_farm (trigger: interval[0:04:55.353196], next run at: 2024-03-23 03:11:10 MDT)" raised an 
exception
Traceback (most recent call last):
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 202, in 
activate_farm_list_raids_for
    list_element = WebDriverWait(driver, 9).until(EC.presence_of_element_located((By.XPATH, list_element_xpath)))
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/support/wait.py"
, line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010482c3e8 chromedriver + 4326376
1   chromedriver                        0x00000001048248b0 chromedriver + 4294832
2   chromedriver                        0x0000000104450088 chromedriver + 278664
3   chromedriver                        0x0000000104492a80 chromedriver + 551552
4   chromedriver                        0x00000001044cb4f8 chromedriver + 783608
5   chromedriver                        0x00000001044874e4 chromedriver + 505060
6   chromedriver                        0x0000000104487f5c chromedriver + 507740
7   chromedriver                        0x00000001047ef984 chromedriver + 4077956
8   chromedriver                        0x00000001047f473c chromedriver + 4097852
9   chromedriver                        0x00000001047d6528 chromedriver + 3974440
10  chromedriver                        0x00000001047f5054 chromedriver + 4100180
11  chromedriver                        0x00000001047c7b18 chromedriver + 3914520
12  chromedriver                        0x000000010481565c chromedriver + 4232796
13  chromedriver                        0x00000001048157d8 chromedriver + 4233176
14  chromedriver                        0x0000000104824524 chromedriver + 4293924
15  libsystem_pthread.dylib             0x0000000183d43034 _pthread_start + 136
16  libsystem_pthread.dylib             0x0000000183d3de3c thread_start + 8


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 363, in 
send_troops_to_farm
    activate_farm_list_raids_for(farm_list_index, driver, distance_limit=7, ignore_curr_state=False)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 205, in 
activate_farm_list_raids_for
    list_element = WebDriverWait(driver, 9).until(EC.presence_of_element_located((By.XPATH, list_element_xpath)))
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/support/wait.py"
, line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010482c3e8 chromedriver + 4326376
1   chromedriver                        0x00000001048248b0 chromedriver + 4294832
2   chromedriver                        0x0000000104450088 chromedriver + 278664
3   chromedriver                        0x0000000104492a80 chromedriver + 551552
4   chromedriver                        0x00000001044cb4f8 chromedriver + 783608
5   chromedriver                        0x00000001044874e4 chromedriver + 505060
6   chromedriver                        0x0000000104487f5c chromedriver + 507740
7   chromedriver                        0x00000001047ef984 chromedriver + 4077956
8   chromedriver                        0x00000001047f473c chromedriver + 4097852
9   chromedriver                        0x00000001047d6528 chromedriver + 3974440
10  chromedriver                        0x00000001047f5054 chromedriver + 4100180
11  chromedriver   

Job "incoming_attack (trigger: interval[0:03:12.928064], next run at: 2024-03-23 03:19:31 MDT)" raised an exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_20290/1855441940.py", line 58, in 
incoming_attack
    if 'att1' in img.get_attribute('class'):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webelemen
t.py", line 178, in get_attribute
    attribute_value = self.parent.execute_script(
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver
.py", line 407, in execute_script
    return self.execute(command, {"script": script, "args": converted_args})["value"]
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver
.py", line 347, in execute
    self.error_handler.check_response(response)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/selenium/webdriver/remote/errorhand
ler.py", line 229, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.StaleElementReferenceException: Message: stale element reference: stale element not 
found
  (Session info: chrome=123.0.6312.59); For documentation on this error, please visit: 
https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x000000010482c3e8 chromedriver + 4326376
1   chromedriver                        0x00000001048248b0 chromedriver + 4294832
2   chromedriver                        0x0000000104450088 chromedriver + 278664
3   chromedriver                        0x000000010445ddb8 chromedriver + 335288
4   chromedriver                        0x0000000104455370 chromedriver + 299888
5   chromedriver                        0x0000000104455370 chromedriver + 299888
6   chromedriver                        0x0000000104453c14 chromedriver + 293908
7   chromedriver                        0x0000000104456684 chromedriver + 304772
8   chromedriver                        0x00000001044cbf30 chromedriver + 786224
9   chromedriver                        0x00000001044cb4f8 chromedriver + 783608
10  chromedriver                        0x00000001044874e4 chromedriver + 505060
11  chromedriver                        0x0000000104487f5c chromedriver + 507740
12  chromedriver                        0x00000001047ef984 chromedriver + 4077956
13  chromedriver                        0x00000001047f473c chromedriver + 4097852
14  chromedriver                        0x00000001047d6528 chromedriver + 3974440
15  chromedriver                        0x00000001047f5054 chromedriver + 4100180
16  chromedriver                        0x00000001047c7b18 chromedriver + 3914520
17  chromedriver                        0x000000010481565c chromedriver + 4232796
18  chromedriver                        0x00000001048157d8 chromedriver + 4233176
19  chromedriver                        0x0000000104824524 chromedriver + 4293924
20  libsystem_pthread.dylib             0x0000000183d43034 _pthread_start + 136
21  libsystem_pthread.dylib             0x0000000183d3de3c thread_start + 8

Shutting down scheduler...


<Popen: returncode: None args: 'curl -X POST "http://127.0.0.1:22999/api/shu...>

%%bash
curl -X POST "http://127.0.0.1:22999/api/shutdown" # used to shutdown proxy-manager